In [2]:
import torch
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import time
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from typing import Iterable
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from joblib import dump, load

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor

import sys

In [3]:
site = 'A'

In [4]:
# load the data
data_finetune = pd.read_csv("data_finetune.csv", index_col = 0)

In [5]:
data_finetune

,ts,Month,Day,Hour,instanceID,Wind_speed,TI,Temperature,Power
118992,2020-05-17 17:20:00,5,17,17,A_WTG01,15.113220,14.855643,11.627620,2558.2880
26598,2020-01-31 18:50:00,1,31,18,A_WTG01,13.458100,16.313596,9.734640,2500.3020
288948,2020-11-30 10:20:00,11,30,10,A_WTG01,14.579640,11.682319,7.631262,2559.3960
79368,2020-04-01 20:40:00,4,1,20,A_WTG01,4.605131,14.969815,7.049312,149.0881
108378,2020-05-05 10:30:00,5,5,10,A_WTG01,4.765651,23.030998,8.375311,218.0337
...,...,...,...,...,...,...,...,...,...
127433,2020-05-27 11:40:00,5,27,11,A_WTG06,3.136577,28.075118,13.594430,39.7835
252341,2020-10-19 01:20:00,10,19,1,A_WTG06,4.021086,22.134016,8.378720,129.2706
28691,2020-02-03 04:50:00,2,3,4,A_WTG06,12.679620,14.700496,6.493042,2557.5790
82229,2020-04-05 04:00:00,4,5,4,A_WTG06,6.011343,10.698589,6.955961,442.9386


In [6]:
# load normalization function 
scaler1 = load('scaler1.bin')
scaler2 = load('scaler2.bin')

In [7]:
turbine_count = data_finetune['instanceID'].nunique()

### Define network

In [8]:
class Net(nn.Module):
    def __init__(self, dims: Iterable[int], output_activation: nn.Module = None):
        """Creates a network using ReLUs between layers and no activation at the end

        :param dims (Iterable[int]): tuple in the form of (IN_SIZE, HIDDEN_SIZE, HIDDEN_SIZE2,
            ..., OUT_SIZE) for dimensionalities of layers
        :param output_activation (nn.Module): PyTorch activation function to use after last layer
        """
        super().__init__()
        self.input_size = dims[0]
        self.out_size = dims[-1]
        self.layers = self.make_seq(dims, output_activation)

    @staticmethod
    def make_seq(dims: Iterable[int], output_activation: nn.Module) -> nn.Module:
        """Creates a sequential network using ReLUs between layers and no activation at the end

        :param dims (Iterable[int]): tuple in the form of (IN_SIZE, HIDDEN_SIZE, HIDDEN_SIZE2,
            ..., OUT_SIZE) for dimensionalities of layers
        :param output_activation (nn.Module): PyTorch activation function to use after last layer
        :return (nn.Module): return created sequential layers
        """
        mods = []

        for i in range(len(dims) - 2):
            mods.append(nn.Linear(dims[i], dims[i + 1]))
            mods.append(nn.ReLU())

        mods.append(nn.Linear(dims[-2], dims[-1]))
        if output_activation:
            mods.append(output_activation())
        return nn.Sequential(*mods)
    
    def forward(self, x: Tensor) -> Tensor:
        """Computes a forward pass through the network

        :param x (torch.Tensor): input tensor to feed into the network
        :return (torch.Tensor): output computed by the network
        """
        # Feedforward
        return self.layers(x)


In [9]:
def train(X, y, quantile, net, lr, batch_size, epoch):    
    
    # create tensor dataset
    train = TensorDataset(Tensor(X), Tensor(y))

    # create data loader from dataset
    trainset = DataLoader(train, batch_size = batch_size, shuffle = True)

    # define optimizer
    optimizer = optim.Adam(net.parameters(), lr = lr)
        
    mse_loss = nn.MSELoss()

    for ep in range(epoch):

        for t in trainset:
            X_temp, y_temp = t
            output = net(X_temp)
            residual = y_temp - output
            loss = Tensor.max(quantile*residual, (quantile-1)*residual).mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    return net, net.state_dict()

### Fine tuning

use a very low learning rate at this stage, because we are training on a dataset that is very small. This is to prevent the risk of overfitting very quickly if we apply large weight updates. 

In [10]:
# load hyperparameters
model = torch.load(sys.path[0] + '/hparams_finetune.pth')

# load the pretrained weights
pretrain = torch.load(sys.path[0] + '/pretrain.pth')

In [11]:
pretrain

{'median': OrderedDict([('layers.0.weight',
               tensor([[-0.2058,  0.1574,  0.0526],
                       [-0.5071, -0.1184, -0.0233],
                       [ 0.1560, -0.1836, -0.0684],
                       [-0.0306, -0.0834,  0.1948],
                       [-0.1010,  0.0483, -0.3136],
                       [-0.5427,  0.1885, -0.0202],
                       [ 0.6917,  0.0757, -0.0173],
                       [ 0.4790,  0.1916,  0.1720],
                       [ 0.5465, -0.0473, -0.1253],
                       [-0.4558, -0.5142,  0.0997],
                       [ 0.0210,  0.2531, -0.0929],
                       [-0.6486, -0.0973, -0.0132],
                       [-0.1934,  0.2575, -0.1671],
                       [-0.4501,  0.1971,  0.0242],
                       [-0.6083,  0.0146,  0.1262],
                       [-0.4521, -0.0311,  0.0765]])),
              ('layers.0.bias',
               tensor([ 0.3656,  0.3384,  0.0042,  0.1991, -0.4763, -0.8338,  0.6098, -0.

In [12]:
# # define quantiles and hyperparameters
q_median = 0.5
q_upper = 0.975
q_lower = 0.025
dims = model['dims']
lr = model['lr']
batch_size = int(model['batch_size'])
epoch = model['epoch']

In [13]:
print("lr:", lr)
print("batch size:", batch_size)
print("epoch:", epoch)

lr: 0.001
batch size: 500
epoch: 20


In [14]:
%%time
#################################################### training ######################################################### 

turbines = data_finetune.instanceID.unique()
median_state_dict_all = []
UQ_state_dict_all = []
LQ_state_dict_all = []


for ID in turbines:
    
    # select data based on turbine ID
    data_temp = data_finetune[data_finetune['instanceID'] == ID]

    # normalize data
    X = scaler1.transform(data_temp.iloc[:, 5:-1])
    y = scaler2.transform(data_temp.iloc[:, -1:])
    
    # create network and load pretrain weights
    net_median_temp = Net(dims = dims)
    net_median_temp.load_state_dict(pretrain['median'])
    
    net_upper_temp = Net(dims = dims)
    net_upper_temp.load_state_dict(pretrain['UQ'])
    
    net_lower_temp = Net(dims = dims)
    net_lower_temp.load_state_dict(pretrain['LQ'])
    
    # train
    net_median_temp, median_state_dict = train(X=X, y=y, quantile=q_median, net=net_median_temp, 
                                          lr=lr, batch_size=batch_size, epoch=epoch)
    net_upper_temp, UQ_state_dict = train(X=X, y=y, quantile=q_upper, net=net_upper_temp, 
                                          lr=lr, batch_size=batch_size, epoch=epoch)
    net_lower_temp, LQ_state_dict = train(X=X, y=y, quantile=q_lower, net=net_lower_temp, 
                                          lr=lr, batch_size=batch_size, epoch=epoch)
    
    median_state_dict_all.append(median_state_dict)
    UQ_state_dict_all.append(UQ_state_dict)
    LQ_state_dict_all.append(LQ_state_dict)
    
    print('Done', ID)
    

Done A_WTG01
Done A_WTG02
Done A_WTG03
Done A_WTG04
Done A_WTG05
Done A_WTG06
Wall time: 1min 10s


### Save trained network

In [15]:
median_state_dict_all_zip = dict(zip(turbines, median_state_dict_all))
UQ_state_dict_all_zip = dict(zip(turbines, UQ_state_dict_all))
LQ_state_dict_all_zip = dict(zip(turbines, LQ_state_dict_all))

In [18]:
# # # save trained network

# torch.save(median_state_dict_all_zip, sys.path[0] + '/median.pth')
# torch.save(UQ_state_dict_all_zip, sys.path[0] + '/UQ.pth')
# torch.save(LQ_state_dict_all_zip, sys.path[0] + '/LQ.pth')